In [ ]:
!pip install easyocr

In [1]:
!pip install langchain_google_genai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.9/415.9 kB 26.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.10
    Uninstalling google-ai-generativelanguage-0.6.10:
      Successfully uninstalled google-ai-generativelanguage-0.6.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.3 requires google-ai-generativelanguage==0.6.10, but you have google-ai-generativelanguage 0.6.16 which is incompatible.
langchain 0.3.

In [26]:
import easyocr

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'])  

# Perform OCR on an image (replace 'image.jpg' with your file)
results = reader.readtext(r'/kaggle/input/invoives/PNG Images/invoice_6.png')

# Extract only text
extracted_text = [text[1] for text in results]

# Print the extracted text
print(extracted_text)


['ATG Seabrook, LLC', '27 Stard Road', 'AuU', 'Seabrook; NH 03874', '(603) 474-8310: Parts', '1; Service', '2; Sales', '3', 'ADVANIAGE TRUCK GRQUP', '~wE Khowtrucks:', 'Fax (603) 474-0427', 'SeabrookParts@AdvantageTruckNE.com', 'Elght Convenient Locatlons ar Onllne 3t AdvantageTruckNEcom', 'MA:', 'Raynham, Shrewsbury & Westfield NH: Lancaster, Lebanon, Manchester & Beabrook VT: Westminster', 'BILL TO', 'DELIVER TO', 'PALMER GAS & OIL DIO PALMER GAS COMPANY INC -', 'PALMER GAS & OIL DJO PALMER GAS', '29287', 'COMPANY INC - 29287', 'P.O. BOX 98', '13 HALL FARM RD', 'NORTH SALEM, NH 03073', 'ATKINSON, NH 03811', 'P: (603} 681-2244', 'P: (603) 681-2244', 'Ship TO CODE', 'PARTS INVOICE: X501I26539:01', 'IBS: 723339', 'DATE CREATD', 'SHIF VIA', 'DATE INVOTCEE', 'SALESPERSOR', 'UNIT TD', 'VIN', '~ALE TYPE', 'TERWS', 'TUSTOMEZR REEFERENICE', '27/2512025', 'SOUTH 495', '2/25/2025', 'MATT M', 'PIBS', 'IBS', 'pg218', 'ar SHP', 'ary Bio', 'ITEM', 'DESCRIPTION', 'BIN', 'LIST PRICE', 'UNIT PrICE', '

In [27]:
from langchain_google_genai import ChatGoogleGenerativeAI
import pandas as pd
import json

# 🔹 Initialize Gemini API
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=api
)


# 🔹 Define Prompt
prompt = f""" THE EXTRACTED TEXT IS THE TEXT OF AN INVOCE
Extract the following structured data from the OCR text,The Terms can be capital or not you have to figure it out yourself:
- INVOICE (Its shows invoice number it is also not case sensitive)
- DATE CREATED
- VENDOR (company name)
- SALE TYPE
- DELIVER TO
- DESCRIPTION (as a list, should be small and concise)
- Quantity (It is the quantity of the product do not confuse it with any price value, it can  start fron QTY or Qty and may or may not have SHP that is quantity of the product its an integer)
- UNIT PRICE (It is the price of one unit, It can be replaced by its synonym and its not case sensitive)
- EXTD PRICE or Total (It can be replaced by its synonym and its not case sensitive)
Return the data in **valid JSON format**.
Ensure that the response contains only the extracted data as a JSON object.

OCR Text:
{extracted_text}
"""

# 🔹 Call Gemini API
response = llm.invoke(prompt)
response = response.content  # Extracts the actual text from AIMessage
print(response)


```json
{
  "INVOICE": "X501I26539:01",
  "DATE CREATED": "2/25/2025",
  "VENDOR": "ADVANIAGE TRUCK GRQUP",
  "SALE TYPE": "PARTS",
  "DELIVER TO": "PALMER GAS & OIL DIO PALMER GAS COMPANY INC - 29287\n13 HALL FARM RD\nATKINSON, NH 03811",
  "DESCRIPTION": [
    "SENSOR,NITROGEN OXIDE",
    "COREBIN",
    "SENSOR,NITROGEN OXI"
  ],
  "Quantity": [
    "1",
    "1"
  ],
  "UNIT PRICE": [
    "596.70",
    "297.50"
  ],
  "EXTD PRICE": [
    "596.70",
    "297.50"
  ]
}
```


In [28]:
import pandas as pd
import json
import re
import os

csv_path = r"/kaggle/working/extracted_data.csv"  # Path for the CSV file

def process_response(response):
    """Extracts structured JSON from Gemini response and saves it to CSV (append if exists)."""
    
    # 🔹 Extract JSON from Response (remove markdown code block)
    response_text = response.content if hasattr(response, "content") else str(response)
    json_match = re.search(r"```json\n(.*?)\n```", response_text, re.DOTALL)
    
    if json_match:
        json_string = json_match.group(1)  # Extract JSON part
    else:
        json_string = response_text  # If no code block, assume raw JSON
    
    # 🔹 Convert JSON string to Python Dictionary
    try:
        extracted_json = json.loads(json_string)
    except json.JSONDecodeError:
        print("❌ Error: Invalid JSON format")
        return
    
    # 🔹 Handle Dynamic Columns & Convert to DataFrame
    rows = []
    max_length = max([len(v) if isinstance(v, list) else 1 for v in extracted_json.values()])  

    for i in range(max_length):
        row = {}
        for key, value in extracted_json.items():
            if isinstance(value, list):  # Handle list values
                row[key] = value[i] if i < len(value) else ""
            else:
                row[key] = value  # Keep same value for non-list fields
        rows.append(row)

    df = pd.DataFrame(rows)  # Convert to DataFrame

    # 🔹 Filter out rows where "UNIT PRICE" is NaN or empty
    df = df[df["UNIT PRICE"].notna() & (df["UNIT PRICE"] != "")]
    
    if df.empty:
        print("❌ No valid data to save (all UNIT PRICE values are NaN).")
        return

    # 🔹 Convert "QUANTITY" column to integer (handle non-numeric safely)
    if "Quantity" in df.columns:
        df["Quantity"] = pd.to_numeric(df["Quantity"], errors="coerce").fillna(0).astype(int)

    # 🔹 Check if CSV exists and Append if it does
    if os.path.exists(csv_path):
        df.to_csv(csv_path, mode='a', header=False, index=False)  # Append without writing headers again
        print("✅ Data saved into existing CSV.")
    else:
        df.to_csv(csv_path, index=False)  # Create new file with headers
        print(f"✅ New CSV created and data saved at {csv_path}")

process_response(response)


✅ Data saved into existing CSV.


In [30]:
import pandas as pd

df=pd.read_csv(r"/kaggle/working/extracted_data.csv")
df.head(15)

,INVOICE,DATE CREATED,VENDOR,SALE TYPE,DELIVER TO,DESCRIPTION,Quantity,UNIT PRICE,EXTD PRICE
0,X501I26538:01,2/25/2025,ADVANIAGE TRUCK GROUP,PARTS,PALMER GAS & OIL DIO PALMER GAS COMPANY INC\n2...,SO1DABP N54 CC 675 265,4,19.18,76.72
1,X501I26538:01,2/25/2025,ADVANIAGE TRUCK GROUP,PARTS,PALMER GAS & OIL DIO PALMER GAS COMPANY INC\n2...,SO[D/A06 40578-000,4,44.99,179.96
2,J0052726,02/25/2025,GRANITE STATE GLASS,NaN,"13 Hall Farm Rd:\nAtkinson, NH 03811",Windshield (Left) (pnt bnd),1,112.00,112.00
3,J0052726,02/25/2025,GRANITE STATE GLASS,NaN,"13 Hall Farm Rd:\nAtkinson, NH 03811",Labor 2.10 hours,1,250.00,250.00
4,278822,02/26/2025,SaNEL,Sale,"Bill Morsett\nNORTH\nSALEM, NH 03073",Shcck Absorber Magnum Rear,2,99.27,198.54
5,120735,02/26/2025,SaNEL NAPA,Charge Sale,"Bill Morsett\nBOX 98\nNORTH SALEM, NH 03073",NAPA Motor Oil 5w30 Full (518 ),5,24.00,122.16
6,27838<,02/26/2025,NAFA,Auto Parts o HD Truck Paint & Body,"NORTH SALEM, NE 63073",MIL 232C ERILL 3IT KII,1,77.06,77.06
7,X501I26539:01,2/25/2025,ADVANIAGE TRUCK GRQUP,PARTS,PALMER GAS & OIL DIO PALMER GAS COMPANY INC - ...,"SENSOR,NITROGEN OXIDE",1,596.70,596.70
8,X501I26539:01,2/25/2025,ADVANIAGE TRUCK GRQUP,PARTS,PALMER GAS & OIL DIO PALMER GAS COMPANY INC - ...,COREBIN,1,297.50,297.50


In [8]:
import shutil
import os

# Path to Kaggle working directory
kaggle_dir = "/kaggle/working/"

# Delete all files and folders in the working directory
for filename in os.listdir(kaggle_dir):
    file_path = os.path.join(kaggle_dir, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)  # Remove file
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)  # Remove directory
    except Exception as e:
        print(f"❌ Error deleting {file_path}: {e}")

print("✅ Kaggle working directory cleared!")


✅ Kaggle working directory cleared!
